In [1]:
import os
import sys
import csv
import json
import moment
import pymysql
import datetime

import numpy as np
import scipy as sp
import scipy.stats as stats
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict

from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter

%matplotlib inline

## This script can generate the results data file required for plots and tables

Step 1. Set which outcome variable you want to generate data for.

In [2]:
# data_file_name = 'data/pandas_df_v3-1_2020-04-25.pkl'
data_file_name = 'data/pandas_df_v4_2020-04-25.pkl'

df = pd.read_pickle(data_file_name)
_version, date_retrieved = data_file_name.split('_df_')[1].split('.')[0].split('_')
print(_version, date_retrieved)

v4 2020-04-25


In [3]:
# outcome = 'intubated'
# days_to = 'days_to_intubation'

outcome = 'died'
days_to = 'days_to_death'

In [4]:
# NaN indicate people who were not covid positive
covidpos = df.copy()

covidpos = covidpos[~pd.isna(covidpos["days_to_intubation"])]

# remove patients who were intubated before they were diagnosed
print(sum(covidpos["days_to_intubation"] < 0))
covidpos = covidpos[covidpos["days_to_intubation"] >= 0]

# remove patients that died before they were diagnosed
print(sum(covidpos["days_to_death"] < 0))
covidpos = covidpos[covidpos["days_to_death"] >= 0]

# remove patients who were intubated or died more than 90 days later
print(sum(covidpos["days_to_intubation"] < 0))
covidpos = covidpos[covidpos["days_to_intubation"] < 90]
covidpos = covidpos[covidpos["days_to_death"] < 90]

covidpos['age_over_65'] = (covidpos['age'] > 65)+1-1

covidpos.describe()

162
12
0


,pat_mrn_id,intubated,days_to_intubation,died,days_to_death,age,sex,smoker,macula,compl_def,...,race_asian,race_white,race_other,race_declined,eth_hispanic,eth_nonhispanic,eth_declinedother,hx_data,co_data,age_over_65
count,6.393000e+03,6393.000000,6393.000000,6393.000000,6393.000000,6393.000000,6393.000000,2817.000000,6393.000000,6393.000000,...,6393.000000,6393.000000,6393.000000,6393.000000,6393.000000,6393.000000,6393.000000,6393.000000,6393.000000,6393.000000
mean,1.094742e+09,0.075708,19.596903,0.096668,19.983576,57.142709,0.496637,0.256656,0.013765,0.000626,...,0.023620,0.284374,0.278742,0.191303,0.341624,0.369936,0.288440,0.629282,0.680119,0.375411
std,1.318007e+08,0.264551,11.343829,0.295529,11.176388,19.885933,0.500028,0.436865,0.116523,0.025008,...,0.151873,0.451151,0.448416,0.393358,0.474291,0.482825,0.453072,0.483035,0.466467,0.484267
min,1.000011e+09,0.000000,0.000000,0.000000,0.000000,0.002738,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.004369e+09,0.000000,11.000000,0.000000,11.000000,41.470226,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.009216e+09,0.000000,20.000000,0.000000,20.000000,58.527036,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,1.200197e+09,0.000000,28.000000,0.000000,28.000000,72.005476,1.000000,1.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.400038e+09,1.000000,84.000000,1.000000,83.000000,120.202601,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Iterate through each of the covariates to model and save the summary for later plotting

Covariates
- macula
- compl_def
- coagulation
- hypertension
- type2_diabetes
- obesity
- cad
- age over 65
- cough

In [122]:
covariates = ['macula', 'age_over_65', 'refctrl', 'compl_def', 'coagulation', 'hypertension', 'type2_diabetes', 'obesity', 'cad', 'smoker']

results = defaultdict(dict)

for covar in covariates:
    
    # univariate analysis
    covar_df = covidpos[[outcome, days_to, covar]]
    cph = CoxPHFitter()
    cph.fit(covar_df.dropna(), duration_col=days_to, event_col=outcome)
    cph.print_summary()
    
    results[covar]['univariate'] = cph.summary.T.to_dict()[covar]
    
    # age and sex corrected analysis
    covar_df = covidpos[[outcome, days_to, covar, 'age', 'sex']]
    cph = CoxPHFitter()
    cph.fit(covar_df.dropna(), duration_col=days_to, event_col=outcome)
    cph.print_summary()
    
    results[covar]['age_sex_corrected'] = cph.summary.T.to_dict()[covar]

In [123]:
# print(json.dumps(results, indent=2))

In [107]:
# save results to file
outfh = open('results/coxph_%s_%s_%s.json' % (outcome, _version, date_retrieved), 'w')
outfh.write(json.dumps(results, indent=2))
outfh.close()